# Day 17: Chronospatial Computer

[*Advent of Code 2024 day 17*](https://adventofcode.com/2024/day/17) and [*solution megathread*](https://redd.it/1hg38ah)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2024/17/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2024%2F17%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


# %load_ext nb_mypy
# %nb_mypy On

In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

# %load_ext pycodestyle_magic
# %pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

In [4]:
part1_example_input = '''Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0'''

In [5]:
def empty_csc():
    return {
        'A': 0, 'B': 0, 'C': 0,
        'Program': (),
        'IC': 0,
        'Output': []
    }

In [6]:
def parse_input(lines):
    csc = empty_csc()
    for line in lines:
        if line.startswith('Register '):
            csc[line[len('Register ')]] = int(line[len('Register X: '):])
        elif line.startswith('Program: '):
            csc['Program'] = tuple(map(int, line[len('Program: '):].split(',')))
    return csc

# csc = parse_input(part1_example_input.splitlines())
csc = parse_input(downloaded['input'].splitlines())

In [7]:
print(csc)

{'A': 66752888, 'B': 0, 'C': 0, 'Program': (2, 4, 1, 7, 7, 5, 1, 7, 0, 3, 4, 1, 5, 5, 3, 0), 'IC': 0, 'Output': []}


In [8]:
# Combo operands 0 through 3 represent literal values 0 through 3.
# Combo operand 4 represents the value of register A.
# Combo operand 5 represents the value of register B.
# Combo operand 6 represents the value of register C.
# Combo operand 7 is reserved and will not appear in valid programs.
def cop(operand, csc):
    if operand <= 3:
        return operand
    elif operand == 4:
        return csc['A']
    elif operand == 5:
        return csc['B']
    elif operand == 6:
        return csc['C']
    else:
        raise ValueError

In [9]:
def instr(opcode, operand, csc):
    # print(f'{opcode=}, {operand=}, {csc['A']=}, {csc['B']=}, {csc['C']=}')
    did_jnz = False
    match opcode:
        case 0 | 6 | 7:
            result = csc['A'] // 2**cop(operand, csc)
            match opcode:
                # The adv instruction (opcode 0) performs division. The numerator is the value in
                # the A register. The denominator is found by raising 2 to the power of the 
                # instruction's combo operand. (So, an operand of 2 would divide A by 4 (2^2); an
                # operand of 5 would divide A by 2^B.) The result of the division operation is 
                # truncated to an integer and then written to the A register.
                case 0:
                    csc['A'] = result
                # The bdv instruction (opcode 6) works exactly like the adv instruction except 
                # that the result is stored in the B register. (The numerator is still read from
                # the A register.)
                case 6:
                    csc['B'] = result
                # The cdv instruction (opcode 7) works exactly like the adv instruction except
                # that the result is stored in the C register. (The numerator is still read from
                # the A register.)
                case 7:
                    csc['C'] = result
        # The bxl instruction (opcode 1) calculates the bitwise XOR of register B and the
        # instruction's literal operand, then stores the result in register B.
        case 1:
            csc['B'] = csc['B'] ^ operand
        # The bst instruction (opcode 2) calculates the value of its combo operand modulo
        # 8 (thereby keeping only its lowest 3 bits), then writes that value to the B 
        # register.
        case 2:
            csc['B'] = cop(operand, csc) % 8
        # The jnz instruction (opcode 3) does nothing if the A register is 0. However, if
        # the A register is not zero, it jumps by setting the instruction pointer to the
        # value of its literal operand; if this instruction jumps, the instruction pointer
        # is not increased by 2 after this instruction.
        case 3:
            if csc['A'] != 0:
                csc['IC'] = operand
                did_jnz = True
        # The bxc instruction (opcode 4) calculates the bitwise XOR of register B and 
        # register C, then stores the result in register B. (For legacy reasons, this 
        # instruction reads an operand but ignores it.)
        case 4:
            csc['B'] = csc['B'] ^ csc['C']
        # The out instruction (opcode 5) calculates the value of its combo operand modulo
        # 8, then outputs that value. (If a program outputs multiple values, they are 
        # separated by commas.)
        case 5:
            csc['Output'].append(cop(operand, csc) % 8)

    if not did_jnz:
        csc['IC'] += 2
    return csc

In [10]:
def run_step(csc):
    assert(csc['IC'] < len(csc['Program']))
    return instr(csc['Program'][csc['IC']], csc['Program'][csc['IC'] + 1], csc)

def run_program(csc):
    while csc['IC'] < len(csc['Program']):
        csc = run_step(csc)
    return tuple(csc['Output'])

print(','.join(map(str, run_program(csc))))

2,0,4,2,7,0,1,0,3


In [11]:
HTML(downloaded['part1_footer'])

In [12]:
HTML(downloaded['part2'])

In [13]:
part2_example_input = '''Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0'''

In [14]:
csc = parse_input(part2_example_input.splitlines())

In [15]:
print(csc)

{'A': 2024, 'B': 0, 'C': 0, 'Program': (0, 3, 5, 4, 3, 0), 'IC': 0, 'Output': []}


In [16]:
csc['A'] = 117440
output = run_program(csc)
print(f'{output=}, {csc['Program']=}')

output=(0, 3, 5, 4, 3, 0), csc['Program']=(0, 3, 5, 4, 3, 0)


In [17]:
def tuple_startswith(slf, value):
    if len(value) > len(slf):
        return False
    else:
        return slf[0:len(value)] == value

In [18]:
def run_program2(csc):
   while csc['IC'] < len(csc['Program']) and tuple_startswith(csc['Program'], tuple(csc['Output'])):
      csc = run_step(csc)
   return csc

In [19]:
csc = parse_input(part2_example_input.splitlines())
csc['A'] = 117440
csc = run_program2(csc)
print(f'{tuple(csc['Output']) == csc['Program']}')

True


In [20]:
from copy import deepcopy

def search_iv(initial_csc):
    iv = 0
    csc = initial_csc.copy()
    while tuple(csc['Output']) != csc['Program'] and iv < 2e6:
        iv += 1
        if iv % 100000 == 0:
            print(iv)
        csc = deepcopy(initial_csc)
        csc['A'] = iv
        csc = run_program2(csc)
    print(f'{iv=}, {csc=}')

In [21]:
#search_iv(parse_input(part2_example_input.splitlines()))
search_iv(parse_input(downloaded['input'].splitlines()))

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
iv=2000000, csc={'A': 250000, 'B': 15625, 'C': 15625, 'Program': (2, 4, 1, 7, 7, 5, 1, 7, 0, 3, 4, 1, 5, 5, 3, 0), 'IC': 14, 'Output': [1]}
